In [12]:
import os

def get_project_root() -> str:
    return os.path.abspath(os.path.join(os.getcwd(), "../"))

In [13]:
print(get_project_root())

/Users/maxmartyshov/Desktop/IU/year3/PMDL/Sentiment_Analysis_for_Financial_News


In [14]:
os.chdir(get_project_root())

from pipelines.extract_training_data import extract_latest_loaders

dataloaders = extract_latest_loaders()
train_loader = dataloaders['train']
val_loader = dataloaders['validation']

Authentication token expired; refreshing...
Pipeline artifact: ac158690-4dd2-4c81-9289-4d1f3f12b40c loaded successfully


In [15]:
def get_input_example():
    batch = next(iter(train_loader))

    # Move the batch to CPU if needed (for logging purposes)
    for key in batch:
        batch[key] = batch[key].cpu()

    # Prepare the input example
    return {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"],
        "has_source": batch["has_source"]
    }

In [16]:
import torch.nn as nn
import torch

from transformers import BertModel


class SentimentAnalysisModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', num_labels=3):
        super(SentimentAnalysisModel, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)

        self.linear1 = nn.Linear(self.bert.config.hidden_size + 1, num_labels)

        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, has_source):
        embeddings = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        has_source = has_source.unsqueeze(1) 
        combined_input = torch.cat((embeddings, has_source), dim=1)

        regularized = self.dropout(combined_input)
        logits = self.linear1(regularized)

        return logits


In [17]:
from tqdm import tqdm

import mlflow
import mlflow.pytorch

def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch):
    model.train()
    train_loss = 0.0
    total = 0.

    loop = tqdm(
        enumerate(dataloader, 1),
        total=len(dataloader),
        desc=f"Epoch {epoch}: train",
        leave=True,
    )

    for _, batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        has_source = batch['has_source'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        logits = model(input_ids = input_ids, attention_mask=attention_mask, has_source=has_source)

        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * input_ids.size(0)
        total += labels.size(0)

        loop.set_postfix({"loss": train_loss/total})

    avg_train_loss = train_loss / total
    mlflow.log_metric('train_loss', avg_train_loss, step=epoch)


def val_one_epoch(model, dataloader, criterion, device, epoch, best_so_far, ckpt_name='model'):
    model.eval()
    val_loss = 0.
    correct = 0.
    total = 0.
    with torch.no_grad():
        loop = tqdm(
            enumerate(dataloader, 1),
            total=len(dataloader),
            desc=f"Epoch {epoch}: val",
            leave=True,
        )
        for i, batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            has_source = batch['has_source'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask, has_source=has_source)

            loss = criterion(logits, labels)
            val_loss += loss.item() * input_ids.size(0)

            _, preds = torch.max(logits, dim=1)
            correct += (preds == labels).sum().item()

            total += labels.size(0)

            loop.set_postfix({"loss": val_loss/total, "acc": correct / total})
        current_acc = correct / total

        avg_val_loss = val_loss / total
        mlflow.log_metric('validation_loss', avg_val_loss, step=epoch)
        mlflow.log_metric('validation_accuracy', current_acc, step=epoch)


        if current_acc > best_so_far:
            print(f"Validation accuracy improved from {best_so_far:.4f} to {current_acc:.4f}. Saving model...")
            mlflow.pytorch.log_model(model, ckpt_name)

            best_so_far = current_acc
    return best_so_far

In [18]:
from mlflow.tracking import MlflowClient

def register_model(run_id, model_name, description):
    """
    Registers the incoming model from the specified run without modifying the Champion tag.
    
    Parameters:
    - run_id: str, the ID of the run where the model is logged.
    - model_name: str, the name of the model in the registry.
    - description: str, a description for the model version.
    
    Returns:
    - version: int, the version of the registered model.
    """
    client = MlflowClient()
    model_uri = f"runs:/{run_id}/{model_name}"
    result = mlflow.register_model(model_uri, model_name)
    print(f"Model registered with name '{model_name}' and version '{result.version}'")
    client.update_model_version(
        name=model_name,
        version=result.version,
        description=description,
    )
    return result.version


In [19]:
def update_champion_alias(model_name, metric_name="validation_accuracy"):
    """
    Goes through all model versions, checks their metrics, and assigns the "Champion" alias to the best-performing model.
    
    Parameters:
    - model_name: str, the name of the model in the MLflow Model Registry.
    - metric_name: str, the metric to base the Champion selection on (default is 'accuracy').
    
    Returns:
    - champion_version: int, the version of the model that is now assigned the "Champion" alias.
    """
    client = MlflowClient()
    
    # Search for all registered versions of the model
    versions = client.search_model_versions(f"name='{model_name}'")
    
    # Initialize variables to track the best version based on the metric
    best_version = None
    best_metric_value = -float('inf')  # Assume we're maximizing the metric (e.g., accuracy)
    best_run_id = None

    # Go through all versions to find the one with the best metric
    for version in versions:
        run_id = version.run_id
        # Get the run's metrics
        run = client.get_run(run_id)
        
        if metric_name in run.data.metrics:
            metric_value = run.data.metrics[metric_name]
            if metric_value > best_metric_value:
                best_metric_value = metric_value
                best_version = version.version
                best_run_id = run_id
    
    # Check if a best version was found
    if best_version is None:
        raise ValueError(f"No models found with metric '{metric_name}'")

    # Reassign the "champion" alias to the best version
    client.set_registered_model_alias(
        name=model_name,
        alias="champion",
        version=best_version
    )
    print(f"Model version {best_version} from run {best_run_id} assigned as 'champion' with {metric_name}: {best_metric_value}")

    return best_version


In [21]:
import torch.optim as optim
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
project_root = os.path.abspath(os.getcwd())
mlflow.set_tracking_uri(f"file://{project_root}/mlruns")

epochs = 2
device = 'mps'
model_name = 'simple_sentiment_analysis_model'
lr = 2e-5

model_desctiption = "BERT, 1 fc layer, 0.3 dropout, ROMA"

model = SentimentAnalysisModel(bert_model_name='bert-base-uncased', num_labels=3).to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=lr)

best_so_far = 0.
mlflow.set_experiment("SentimentAnalysis")

with mlflow.start_run():
    mlflow.log_param("learning_rate", 2e-5)
    mlflow.log_param("epochs", epochs)
    run = mlflow.active_run()
    run_id = run.info.run_id
    for epoch in range(epochs):
        train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)
        best_so_far = val_one_epoch(model, val_loader, criterion, device, epoch, best_so_far, model_name)
    register_model(run_id, model_name, model_desctiption)
    update_champion_alias(model_name)

2024/10/23 13:24:28 INFO mlflow.tracking.fluent: Experiment with name 'SentimentAnalysis' does not exist. Creating a new experiment.
Epoch 0: val: 100%|██████████| 52/52 [00:09<00:00,  5.72it/s, loss=0.625, acc=0.765]


Validation accuracy improved from 0.0000 to 0.7649. Saving model...


2024/10/23 13:25:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Epoch 1: val: 100%|██████████| 52/52 [00:09<00:00,  5.53it/s, loss=0.487, acc=0.824]


Validation accuracy improved from 0.7649 to 0.8240. Saving model...


2024/10/23 13:26:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model registered with name 'simple_sentiment_analysis_model' and version '1'
Model version 1 from run b7ffbb627ea9491f82c84506b92b78fa assigned as 'champion' with validation_accuracy: 0.8239903556359253


Successfully registered model 'simple_sentiment_analysis_model'.
Created version '1' of model 'simple_sentiment_analysis_model'.


In [22]:
mlflow.get_tracking_uri()

'file:///Users/maxmartyshov/Desktop/IU/year3/PMDL/Sentiment_Analysis_for_Financial_News/mlruns'